# Notebook 03 — Wikipedia Circuits Exploration  

## 1. Présentation de la source : Wikipedia

### Objectif de la section

Cette section vise à présenter la source de données utilisée dans ce notebook, à en définir précisément le rôle dans l’architecture du projet *F1 Predictive Assistant (F1PA)*, et à justifier son usage dans une démarche data professionnelle structurée selon une architecture Bronze / Silver / Gold.

L’objectif est de fournir un cadre clair et défendable permettant de comprendre :
- pourquoi Wikipedia est utilisée,
- pour quel type de données,
- avec quelles limites méthodologiques assumées.


### Rôle de Wikipedia dans le projet F1 Predictive Assistant

Dans le cadre du projet *F1 Predictive Assistant (F1PA)*, Wikipedia est utilisée comme source de données pour les **circuits de Formule 1**, exclusivement à des fins de **référencement géographique**.

Les circuits constituent des entités statiques ou faiblement évolutives, dont les caractéristiques principales (nom, localisation, coordonnées géographiques) sont :
- stables dans le temps,
- indépendantes de la performance sportive instantanée,
- nécessaires pour contextualiser d’autres sources de données du projet (OpenF1 et Meteostat).

Wikipedia offre un accès libre à ce type d’informations, généralement présentées de manière structurée via les *infobox* des pages dédiées aux circuits.


### Données recherchées dans Wikipedia

Les informations extraites depuis Wikipedia dans ce notebook sont volontairement limitées aux métadonnées géographiques suivantes :

- nom officiel du circuit,
- pays,
- ville ou zone géographique,
- latitude,
- longitude.

Ces données constituent le socle géographique minimal nécessaire pour :
- sélectionner ultérieurement des stations météo pertinentes,
- contextualiser les données météorologiques issues de Meteostat,
- enrichir les données OpenF1 en couche Silver.

Aucune information sportive, historique ou événementielle n’est exploitée à ce stade.

### Positionnement architectural des données (couche Bronze)

Les données issues de Wikipedia sont intégrées exclusivement en **couche Bronze**, selon les principes suivants :

- données brutes, non enrichies,
- absence de correction manuelle ou d’inférence externe,
- absence totale de jointure avec d’autres sources,
- traçabilité explicite de la source et de la date de scraping.

Ce positionnement est cohérent avec :
- le caractère non contractuel de Wikipedia,
- la vocation exploratoire et préparatoire de la couche Bronze.

Toute transformation, normalisation ou enrichissement sera explicitement reporté aux couches Silver et Gold du projet.

### Limites connues et posture critique

L’utilisation de Wikipedia comme source de données implique plusieurs limites connues :

- données maintenues de manière collaborative,
- absence de garantie formelle sur l’exhaustivité ou la fréquence de mise à jour,
- possibles incohérences de format ou de granularité selon les pages.

Ces limites sont assumées et documentées. Elles justifient :
- la réalisation d’une analyse de qualité dédiée dans ce notebook,
- une utilisation strictement limitée à des données de référence statiques,
- l’exclusion de toute donnée critique ou dynamique du périmètre Wikipedia.

### Conclusion de la section

Wikipedia est utilisée dans ce notebook comme une source de référence géographique adaptée à une intégration en couche Bronze.

Cette approche garantit :
- une cohérence architecturale avec les notebooks précédents,
- une traçabilité claire des décisions techniques

---------

## 2. Méthodologie de scraping

### Objectif de la méthodologie

L’objectif de cette section est de définir une stratégie de scraping claire, reproductible et maîtrisée afin d’extraire des données géographiques fiables sur les circuits de Formule 1 à partir de Wikipedia.

Cette méthodologie doit répondre aux contraintes suivantes :
- utilisation exclusive de Wikipedia comme source,
- extraction de données structurées et traçables,
- gestion explicite des cas incomplets ou incohérents,
- conformité avec une intégration en couche Bronze.

### Choix des pages Wikipedia

La stratégie retenue repose sur une approche en deux étapes :

1. Identification d’une page de référence listant les circuits de Formule 1.
2. Accès aux pages individuelles de chaque circuit afin d’extraire les informations géographiques depuis les infobox.

Ce choix permet :
- de limiter les hypothèses implicites sur la structure des données,
- de s’appuyer sur des pages dédiées à chaque entité circuit,
- de gérer les variations de format ou de complétude page par page.

### Source principale utilisée

La page de référence utilisée comme point d’entrée est :

- la liste des circuits de Formule 1 disponible sur Wikipedia.

Cette page permet d’identifier l’ensemble des circuits ayant accueilli des Grands Prix de Formule 1, sans distinction initiale de période.

Le filtrage temporel (2022–2025) n’est volontairement pas appliqué à ce stade, afin de :
- conserver une vision exhaustive des circuits,
- reporter toute logique de sélection métier à une étape ultérieure.

### Stratégie de scraping retenue

La stratégie de scraping adoptée repose sur une approche en deux étapes :

1. **Extraction de la liste des circuits** depuis la page centrale :
   - Nom officiel du circuit
   - URL de la page Wikipedia du circuit
   - Ville / localité
   - Pays

2. **Enrichissement circuit par circuit** à partir de chaque page individuelle afin de récupérer :
   - Latitude
   - Longitude

En cas d’absence d’une information :
- la valeur est laissée manquante,
- aucune imputation ou correction n’est réalisée en couche Bronze.

Cette approche est volontairement choisie car :
- la page de liste fournit déjà des informations propres et bien structurées (ville / pays séparés),
- les coordonnées géographiques ne sont pas systématiquement présentes dans le tableau principal,
- elle garantit une meilleure robustesse que le parsing de champs textuels composites.

Toutes les données sont collectées telles quelles, sans transformation métier, conformément aux principes de la couche Bronze.

### Outils techniques utilisés

Le scraping est réalisé en Python à l’aide des bibliothèques suivantes :

- `requests` pour la récupération des pages HTML,
- `BeautifulSoup` pour le parsing du contenu,
- `pandas` pour la structuration des données extraites.

Un en-tête HTTP explicite (User-Agent) est utilisé afin de :
- respecter les bonnes pratiques de scraping,
- assurer une transparence vis-à-vis de la source.

### Gestion des erreurs et limites

La méthodologie intègre une gestion explicite des cas suivants :

- pages indisponibles ou inaccessibles,
- infobox absente ou partiellement structurée,
- champs géographiques manquants ou non standardisés.

Ces cas sont :
- documentés,
- conservés tels quels en Bronze,
- analysés dans la section dédiée à la qualité des données.

-------------

## 3. Scrapping

### Imports

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
from pathlib import Path

### Initialisation du scrapping

In [2]:
HEADERS = {
    "User-Agent": "F1PredictiveAssistant/1.0 (Academic project, data exploration)"
}

SCRAPING_DATE = datetime.utcnow().strftime("%Y-%m-%d")

In [3]:
WIKIPEDIA_CIRCUITS_URL = "https://en.wikipedia.org/wiki/List_of_Formula_One_circuits"

In [4]:
response = requests.get(WIKIPEDIA_CIRCUITS_URL, headers=HEADERS)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

In [5]:
soup.title.text

'List of Formula One circuits - Wikipedia'

### Extraction des infos circuits

In [6]:
tables = soup.find_all("table", {"class": "wikitable"})
print(f"Nombre de tables trouvées : {len(tables)}")

Nombre de tables trouvées : 2


La page Wikipedia contient plusieurs tables de type `wikitable`.

- La première table correspond uniquement à une légende (circuits actuels / futurs).
- La seconde table, intitulée **"Formula One circuits"**, contient la liste complète des circuits utilisés en F1.

Seule cette seconde table est pertinente pour notre extraction.

In [7]:
circuits_table = tables[1]

rows = circuits_table.find("tbody").find_all("tr")[1:]

print(f"Nombre de lignes (circuits) trouvées : {len(rows)}")

Nombre de lignes (circuits) trouvées : 78


À partir de la table principale, nous extrayons les informations suivantes :

- **Nom du circuit**  
- **Lien vers la page Wikipedia du circuit**  
- **Ville / localité**  
- **Pays**

Ces informations sont déjà correctement séparées dans le tableau, ce qui évite toute logique de parsing fragile
sur des chaînes de caractères composites.

Aucune information géographique fine (latitude / longitude) n’est encore récupérée à ce stade.

In [8]:
circuits_list = []

BASE_WIKI_URL = "https://en.wikipedia.org"

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 6:
        continue

    # Nom du circuit + URL
    circuit_link = cols[0].find("a")
    circuit_name = circuit_link.get_text(strip=True) if circuit_link else None
    circuit_url = BASE_WIKI_URL + circuit_link["href"] if circuit_link else None

    # Localité (ville)
    location_links = cols[4].find_all("a")
    locality = location_links[0].get_text(strip=True) if location_links else None

    # Pays
    country_links = cols[5].find_all("a")
    country = country_links[-1].get_text(strip=True) if country_links else None

    circuits_list.append({
        "circuit_name": circuit_name,
        "circuit_url": circuit_url,
        "locality": locality,
        "country": country
    })

print(f"Circuits extraits depuis la liste : {len(circuits_list)}")
circuits_list[:3]

Circuits extraits depuis la liste : 78


[{'circuit_name': 'Adelaide Street Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Adelaide_Street_Circuit',
  'locality': 'Adelaide',
  'country': 'Australia'},
 {'circuit_name': 'Ain-Diab Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Ain-Diab_Circuit',
  'locality': 'Casablanca',
  'country': 'Morocco'},
 {'circuit_name': 'Aintree Motor Racing Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Aintree_Motor_Racing_Circuit',
  'locality': 'Aintree',
  'country': 'United Kingdom'}]

### Structuration tabulaire de la liste des circuits

La première étape de scraping a permis de construire une liste de dictionnaires
Python, chaque entrée représentant un circuit de Formule 1 identifié sur la page
de référence Wikipedia.

Afin de faciliter les étapes d’enrichissement, d’analyse de qualité et de
stockage en couche Bronze, cette structure est convertie en DataFrame pandas.
Cette opération ne modifie pas les données extraites, elle formalise simplement
leur représentation sous forme tabulaire.

In [9]:
circuits_df = pd.DataFrame(circuits_list)

circuits_df

,circuit_name,circuit_url,locality,country
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal
...,...,...,...,...
73,TI Circuit Aida,https://en.wikipedia.org/wiki/Okayama_Internat...,Mimasaka,Japan
74,Valencia Street Circuit,https://en.wikipedia.org/wiki/Valencia_Street_...,Valencia,Spain
75,Watkins Glen International,https://en.wikipedia.org/wiki/Watkins_Glen_Int...,Watkins Glen,United States
76,Yas Marina Circuit,https://en.wikipedia.org/wiki/Yas_Marina_Circuit,Abu Dhabi,United Arab Emirates


### Enrichissement géographique : latitude et longitude

À ce stade, chaque circuit est identifié par son nom et par l’URL de sa page
Wikipedia individuelle. Ces pages contiennent, dans la majorité des cas, un
bloc *infobox* incluant les coordonnées géographiques du circuit.

L’objectif de cette étape est d’extraire, lorsque l’information est disponible :

- la latitude
- la longitude

Ces deux variables constituent le socle minimal nécessaire à toute
contextualisation géographique ultérieure (sélection de stations météo,
calcul de distance, agrégation régionale), sans implémenter ici de logique
métier.

L’extraction repose exclusivement sur le HTML de Wikipedia, sans appel à des
API externes.

La latitude et la longitude ne sont pas présentes dans la table principale.

Nous allons donc visiter chaque page individuelle de circuit afin de récupérer les coordonnées
géographiques depuis les balises HTML standard de Wikipedia (`geo-dec`, `latitude`, `longitude`).

Les coordonnées sont présentées sous deux formats :

- **DMS** (degrés, minutes, secondes) via la classe `geo-dms`
- **Décimal** via la classe `geo-dec`

Dans une optique data science et modélisation, le format décimal est retenu
en priorité. Il est directement exploitable numériquement et ne nécessite
aucune transformation supplémentaire.

Lorsque le format décimal est absent (cas rares), l’extraction échoue
volontairement afin de préserver la qualité et l’homogénéité de la couche Bronze.

In [10]:
def extract_coordinates(url, headers):
    """
    Extrait les coordonnées (latitude, longitude) depuis une page Wikipedia.
    
    Paramètres :
        url (str) : URL de la page Wikipedia.
        headers (dict) : En-têtes HTTP à utiliser pour la requête.
        
    Retour :
        (lat, lon) (tuple de float) : Coordonnées décimales, ou (None, None) si non trouvées.
    """
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException:
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")
    
    # 1. Récupération coordonnées décimales cachées via classe geo
    geo = soup.select_one("table.infobox span.geo-inline span.geo-nondefault span.geo")
    if geo:
        try:
            lat_str, lon_str = geo.text.split(";")
            return float(lat_str.strip()), float(lon_str.strip())
        except ValueError:
            return None, None

    # 2. Sinon récupération coordonnées DMS affichées via classe geo-dev + traitement
    geo_dec = soup.select_one("table.infobox span.geo-inline span.geo-default span.geo-dec")
    if geo_dec:
        try:
            lat_txt, lon_txt = geo_dec.text.strip().split()
            def parse_dms(val):
                sign = -1 if val[-1] in ("S", "W") else 1
                return sign * float(val[:-2].replace("°",""))
            return parse_dms(lat_txt), parse_dms(lon_txt)
        except Exception:
            return None, None

    # 3. Pas de coordonnées trouvées
    return None, None

In [11]:
latitudes = []
longitudes = []

for _, row in circuits_df.iterrows():
    url = row["circuit_url"]
    lat, lon = extract_coordinates(url, HEADERS)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(1)

In [12]:
circuits_df["latitude"] = latitudes
circuits_df["longitude"] = longitudes

In [13]:
circuits_df.head(10)

,circuit_name,circuit_url,locality,country,latitude,longitude
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia,-34.93056,138.62056
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco,33.57861,-7.68750
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom,53.47694,-2.94056
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia,-37.84972,144.96833
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal,37.23194,-8.63194
5,Autódromo do Estoril,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_d...,Estoril,Portugal,38.75083,-9.39417
6,Autódromo Hermanos Rodríguez,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_H...,Mexico City,Mexico,19.40611,-99.09250
7,Autódromo Internacional do Rio de Janeiro,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_I...,Rio de Janeiro,Brazil,-22.97556,-43.39500
8,Autodromo Internazionale del Mugello,https://en.wikipedia.org/wiki/Autodromo_Intern...,Scarperia e San Piero,Italy,43.99750,11.37194
9,Autodromo Internazionale Enzo e Dino Ferrari,https://en.wikipedia.org/wiki/Imola_Circuit,Imola,Italy,44.34111,11.71333


----------------

## 4. Vérification et validation de la table circuits

À ce stade du notebook, nous disposons d’une table `circuits_df` construite exclusivement à partir de données scrapées depuis Wikipedia.  
Cette table contient pour chaque circuit :

- son nom officiel,
- l’URL de sa page Wikipedia (traçabilité),
- sa localisation (ville et pays),
- ses coordonnées géographiques (latitude, longitude).

Avant toute analyse ou décision ETL, il est indispensable de procéder à une **validation minimale des données**, conforme aux principes de la couche Bronze :

- vérifier la structure du DataFrame,
- identifier les valeurs manquantes,
- détecter d’éventuelles incohérences évidentes.

Aucune correction, enrichissement ou jointure n’est réalisée à ce stade.

### Aperçu général

In [15]:
circuits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   circuit_name  78 non-null     object 
 1   circuit_url   78 non-null     object 
 2   locality      78 non-null     object 
 3   country       78 non-null     object 
 4   latitude      77 non-null     float64
 5   longitude     77 non-null     float64
dtypes: float64(2), object(4)
memory usage: 3.8+ KB


### Vérification des valeurs manquantes

Les données géographiques (latitude et longitude) jouent un rôle structurant dans la suite du projet, notamment pour la sélection et l’alignement des stations météo en couche Silver.

Dans une logique Bronze :
- les valeurs manquantes sont **acceptées**,
- mais elles doivent être **explicitement identifiées et documentées**.

L’objectif ici n’est pas de corriger, mais de qualifier la complétude des données issues du scraping Wikipedia.

In [16]:
circuits_df.isna().sum().sort_values(ascending=False)

latitude        1
longitude       1
circuit_url     0
circuit_name    0
country         0
locality        0
dtype: int64

In [17]:
circuits_df[circuits_df["latitude"].isna() | circuits_df["longitude"].isna()]

,circuit_name,circuit_url,locality,country,latitude,longitude
19,Caesars Palace Grand Prix Circuit,https://en.wikipedia.org/wiki/Caesars_Palace_G...,Paradise,United States,NaN,NaN


Nous pouvons voir qu'il nous manque les coordonnées pour le circuit du Caesars Palace. Après vérification sur sa page Wikipédia, nous remarquons que ses latitudes et longitudes ne sont pas renseignées, il ne s'agit donc pas d'une erreur dans le code.  

Ce circuit nécessitera une décision ultérieure :  
- exclusion,
- enrichissement manuelle,
- ou fallback géographique  

Les autres données semblent être bien renseignées.

### Contrôle de cohérence géographique

Avant d'utiliser ces coordonnées comme référence géographique, il faut vérifier qu'elles soient plausibles, cohérentes, et non erronnées.  

Les valeurs doivent respecter ces bornes, sinon il s'agirait d'une erreur de scraping, de parsing, ou même de renseignement :  
- Latitude ∈ [-90 ; 90]
- Longitude ∈ [-180 ; 180]

In [18]:
invalid_coords = circuits_df[
    (circuits_df["latitude"].abs() > 90) |
    (circuits_df["longitude"].abs() > 180)
]

invalid_coords

,circuit_name,circuit_url,locality,country,latitude,longitude


Les coordonnées semblent ne montrer aucune incohérences.

### Vérification des doublons logiques

Un circuit de Formule 1 correspond à une localisation géographique unique.  
Toute duplication stricte de coordonnées peut indiquer :

- un changement de nom au fil du temps,
- une redondance de pages Wikipedia,
- ou une ambiguïté historique.

Ce type de situation n’est **pas résolu ici** et sera traité, si nécessaire, dans les couches supérieures.

In [19]:
circuits_df.duplicated(
    subset=["latitude", "longitude"],
    keep=False
).sum()

np.int64(0)

--------------

## 5. Rôle des données

### Rôle des circuits dans la prédiction du temps au tour

Dans le cadre du projet *F1 Predictive Assistant*, les circuits ne constituent pas des features prédictives directes, mais une **table de référence géographique**.

Leur rôle est central pour :
- contextualiser les données météo,
- sélectionner les stations météorologiques pertinentes,
- garantir une cohérence spatiale dans les couches Silver et Gold.

La qualité de cette table conditionne donc indirectement la performance du modèle final.

### Latitude et longitude : variables structurelles

La latitude et la longitude ne sont pas utilisées comme variables explicatives du modèle dans ce notebook.

Elles servent exclusivement à :
- ancrer chaque circuit dans l’espace,
- préparer l’alignement futur avec les données Meteostat,
- garantir la traçabilité géographique des décisions.

Aucune transformation ni normalisation n’est appliquée à ce stade.

---------

## 6. Décisions ETL pour la couche Bronze, et transition vers la couche Silver

Les décisions suivantes sont appliquées dans ce notebook :

- scraping exclusivement depuis Wikipedia,
- conservation des données brutes,
- absence de correction manuelle,
- acceptation des valeurs manquantes,
- traçabilité via l’URL source.

Ce notebook ne réalise :
- aucune jointure,
- aucun feature engineering,
- aucun calcul métier.

La table `circuits` produite constitue une **table de référence géographique**.

Elle sera utilisée ultérieurement pour :
- l’alignement avec les données Meteostat,
- la sélection des stations météo,
- l’enrichissement des données OpenF1.

Aucune logique d’alignement spatial ou temporel n’est implémentée ici, conformément au découpage Bronze / Silver / Gold.

--------------

## 7. Sauvegarde de la table circuits

À l’issue de l’exploration et du scraping, la table de référence `circuits` est sauvegardée dans la couche Bronze.

Cette sauvegarde permet de :
- figer l’état des données issues de Wikipedia,
- garantir la reproductibilité des traitements,
- éviter le re-scraping systématique de la source,
- assurer une séparation claire entre exploration (Bronze) et enrichissement métier (Silver).

Aucune transformation supplémentaire n’est appliquée à ce stade.

In [21]:
BRONZE_DIR = Path("../data/bronze")
BRONZE_DIR.mkdir(parents=True, exist_ok=True)

output_path = BRONZE_DIR / "circuits_wikipedia_bronze.csv"

# Sauvegarde sans index pour préserver un format tabulaire propre
circuits_df.to_csv(output_path, index=False)

print(f"Table circuits sauvegardée en couche Bronze : {output_path}")

Table circuits sauvegardée en couche Bronze : ..\data\bronze\circuits_wikipedia_bronze.csv


La table est sauvegardée :  
- au format CSV : format lisible, universel, adapté à une table de référence
- sans index : l'identifiant métier n'est pas encore défini
- sans nettoyage : le traitement des données sera abordé en couche Silver

------------

## 8. Conclusion — Wikipedia Circuits Exploration

Ce notebook avait pour objectif d’explorer et structurer les informations relatives aux circuits de Formule 1 à partir d’une source unique : Wikipedia.

Les étapes suivantes ont été réalisées :

- identification des circuits de Formule 1 via la page de référence,
- extraction des informations structurantes (nom, pays, localité),
- enrichissement géographique par scraping individuel des pages circuits,
- construction d’une table de référence `circuits`,
- évaluation de la qualité et des limites des données.

L’ensemble du travail a été mené dans une logique strictement **exploratoire et préparatoire**, sans interaction avec les autres sources du projet.

### Positionnement dans l’architecture Bronze / Silver / Gold

La table `circuits` produite dans ce notebook est considérée comme une **table Bronze** :

- données brutes issues du scraping,
- absence de corrections manuelles,
- valeurs manquantes conservées,
- traçabilité assurée via les URL Wikipedia.

Elle constitue une table de référence géographique, destinée à être enrichie et exploitée ultérieurement en couche Silver.

### Décision d’extraction et persistance des données

Afin de garantir la reproductibilité et la continuité du projet, la table `circuits` est exportée à l’issue de ce notebook.

Cette extraction permet :
- d’isoler clairement la couche Bronze,
- d’éviter le re-scraping systématique de Wikipedia,
- de figer l’état des données à un instant donné,
- de faciliter leur réutilisation contrôlée dans les notebooks suivants.

Le format d’export retenu est volontairement simple et non transformé.

### Choix du format de stockage

La table `circuits` est sauvegardée dans un format tabulaire standard (CSV), avec les caractéristiques suivantes :

- une ligne par circuit,
- des colonnes explicitement nommées,
- aucune transformation métier appliquée,
- compatibilité directe avec les traitements Python ultérieurs.

Ce choix garantit un équilibre entre simplicité, lisibilité et intégration future dans la couche Silver.

### Continuité avec les couches Silver et Gold

Ce notebook marque la fin de la phase d’exploration des circuits.

Les étapes suivantes du projet consisteront à :
- aligner les circuits avec les données météo,
- sélectionner les stations Meteostat pertinentes,
- enrichir les données OpenF1 à des fins de modélisation.

Ces traitements relèvent explicitement des couches Silver et Gold et ne sont volontairement pas abordés ici.